In [103]:
% matplotlib inline
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db = MySQLdb.connect(host="crunchbase.ckfkhzm7cot3.us-west-2.rds.amazonaws.com",
                     port=3306,
                     user="root",         # your username
                     passwd="root1234",  # your password
                     db="analytics_2")        # name of the data base


In [ ]:
cur = db.cursor()

In [ ]:
count = cur.execute("SELECT count(*) FROM cb_objects")

#### Load into pandas dataframe from database

In [ ]:
df_mysql = pd.read_sql('select * from cb_objects LIMIT 100;', con=db)

In [ ]:
persons = pd.read_sql('Select * from cb_objects where entity_type = \'Person\'', con=db)

In [ ]:
relationships = pd.read_sql('Select * from cb_relationships', con=db)

In [ ]:
degrees = pd.read_sql('Select * from cb_degrees', con=db)

#### Fill nan for empty and None values

In [106]:
persons.fillna(value=np.nan, inplace=True)
relationships.fillna(value=np.nan, inplace=True)
degrees.fillna(value=np.nan, inplace=True)

#### Work Experience

In [ ]:
work_ex = pd.merge(persons, relationships, how='outer', right_on='person_object_id', left_on='id')

In [111]:
print 'Total number of work ex rows', len(work_ex)

Total number of work ex rows 428251


In [73]:
workex_with_no_persons = work_ex[work_ex['id_x'].isnull()]
person_with_no_workex = work_ex[work_ex['id_y'].isnull()]
print "workex rows with no corresponding person",len(workex_with_no_persons)
print "Persons with no details about their workex",len(person_with_no_workex)

workex rows with no corresponding person 921
Persons with no details about their workex 25373


In [110]:
print "Number of rows with no start_at for workex is: ",len(work_ex[work_ex['start_at'].isnull()]), "which is roughly half the number of rows"

Number of rows with no start_at for workex is:  221256 which is roughly half the number of rows


Idea : We can fill the start_at date with one of the following:
1. founding_date of the company
2. created_at_y which is when the record was created in the database

To analyse we can first build a plot of the different between the 2 and see the relation between the title, created at and founding date. It can mean that most of the founders have their profile created on crunchbase much later than the founding date, in which case we can safely use the founding date. However, for employess we an use the created_at.

In [112]:
work_ex[:5]

,id_x,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at_y,updated_at_y
0,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,1.0,p:2,c:1,NaN,NaN,0.0,8.0,Co-Founder/CEO/Board of Directors,2007-05-25 07:03:54,2013-06-03 09:58:46
1,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,11366.0,p:2,c:4993,NaN,NaN,1.0,6.0,Co-founder & VP,2008-05-06 00:04:18,2013-06-02 16:56:52
2,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,11367.0,p:2,c:233024,NaN,NaN,1.0,2.0,Business Development,2008-05-06 00:04:18,2013-06-02 16:56:52
3,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,22537.0,p:2,f:1017,NaN,NaN,0.0,7.0,Limited Partner,2008-10-07 02:23:04,2010-10-08 21:02:10
4,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,112873.0,p:2,c:18661,NaN,NaN,1.0,3.0,Consultant,2011-05-11 01:24:28,2011-05-11 01:24:28


#### Education

In [70]:
len(degrees)

109610

In [ ]:
degrees[:5]

In [71]:
len(degrees['object_id'].unique())

68451

We have degree details for the above number of people

In [ ]:
education = pd.merge(persons, degrees, how='outer', left_on='id', right_on='object_id')

In [72]:
education_with_no_persons = education[education['id_x'].isnull()]
person_with_no_education = education[education['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Education rows with no corresponding person 554
Persons with no details about their education 158637


Next Steps: 
1. We have sparse data for education. What to do with it?
2. How to clean workex data?

In [105]:
a = work_ex[:5].groupby('id_x')
work_ex[:10]

,id_x,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at_y,updated_at_y
0,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,1.0,p:2,c:1,NaN,NaN,0.0,8.0,Co-Founder/CEO/Board of Directors,2007-05-25 07:03:54,2013-06-03 09:58:46
1,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,11366.0,p:2,c:4993,NaN,NaN,1.0,6.0,Co-founder & VP,2008-05-06 00:04:18,2013-06-02 16:56:52
2,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,11367.0,p:2,c:233024,NaN,NaN,1.0,2.0,Business Development,2008-05-06 00:04:18,2013-06-02 16:56:52
3,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,22537.0,p:2,f:1017,NaN,NaN,0.0,7.0,Limited Partner,2008-10-07 02:23:04,2010-10-08 21:02:10
4,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,112873.0,p:2,c:18661,NaN,NaN,1.0,3.0,Consultant,2011-05-11 01:24:28,2011-05-11 01:24:28
5,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,260915.0,p:2,c:537,2002-01-01,2002-01-01,1.0,1.0,Consultant,2013-06-02 16:56:52,2013-06-02 16:56:52
6,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,260917.0,p:2,c:2361,NaN,NaN,1.0,4.0,Consultant,2013-06-02 16:56:52,2013-06-02 16:56:52
7,p:2,Person,2.0,NaN,Ben Elowitz,ben elowitz,/person/ben-elowitz,NaN,operating,NaN,...,414818.0,p:2,c:268504,NaN,NaN,1.0,5.0,Consultant,2013-10-01 00:48:10,2013-10-01 00:48:10
8,p:3,Person,3.0,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,2.0,p:3,c:1,NaN,NaN,1.0,279242.0,VP Marketing,2007-05-25 07:04:16,2010-05-21 16:31:34
9,p:3,Person,3.0,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,22539.0,p:3,f:1017,NaN,NaN,0.0,279243.0,Limited Partner,2008-10-07 02:23:04,2010-10-08 21:02:10
